In [10]:
import getpass
import os

# Prompt for API key
#LLAMA_CLOUD_API_KEY = getpass("Enter your LLAMA CLOUD API key: ")
#ANTHROPIC_API_KEY = getpass("Enter your Antropic API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


In [11]:
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Retrieve OpenAI API key from environment variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    logging.error("openai API key not found. Please set the 'OPENAI_API_KEY' environment variable.")
else:
    OPENAI_API_KEY = OPENAI_API_KEY

In [16]:
import os
from openai import OpenAI
import logging

def merge_database_package_procedures_with_history(
        differences_file_path, heritage_package_procedure, geminia_package_procedure, output_package_path):
    """Merges procedures using GPT with history and completion checks using 'FINALLY DONE'."""

    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    
    def chat_with_gpt(prompt, history):
        """Handles communication with GPT model using history."""
        messages = [{"role": "system", "content": "You are an expert PL/SQL developer."}]
        messages.extend(history)
        messages.append({"role": "user", "content": prompt})
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            logging.error(f"OpenAI API Error: {e}")
            raise RuntimeError(f"Failed to communicate with GPT: {e}")

    # Read file contents
    with open(differences_file_path, 'r') as diff_file:
        diff_content = diff_file.read()
    with open(heritage_package_procedure, 'r') as heritage_file:
        heritage_content = heritage_file.read()
    with open(geminia_package_procedure, 'r') as geminia_file:
        geminia_content = geminia_file.read()

    base_prompt = f"""
You are an expert PL/SQL developer. Your task is to merge two PL/SQL procedures into a single harmonized procedure.
The differences between the two procedures are described below (use this for context, do not paste it directly):
{diff_content}
The first procedure (Heritage Procedure):
{heritage_content}
The second procedure (Geminia Procedure):
{geminia_content}
If the merged procedure exceeds 400 lines, generate it in chunks of up to 400 lines. Indicate the completion of the merged procedure with the phrase 'FINALLY DONE' at the end. Otherwise, continue generating chunks.
"""

    merged_chunks = []
    history = []  # Keep track of conversation history
    prompt = base_prompt
    generation_complete = False

    try:
        while not generation_complete:
            chunk = chat_with_gpt(prompt, history)
            history.append({"role": "assistant", "content": chunk})
            if "FINALLY DONE" in chunk.upper():
                generation_complete = True
                chunk = chunk.replace("FINALLY DONE", "").strip()
            merged_chunks.append(chunk)
            logging.info(f"Generated chunk: {len(merged_chunks)} lines.")
            prompt = "Continue generating the merged procedure."
    except Exception as e:
        logging.error(f"LLM Error: {e}")
        raise RuntimeError(f"Failed to merge using GPT: {e}")

    # Combine all chunks
    merged_content = "\n\n".join(merged_chunks)
    output_dir = os.path.dirname(output_package_path) or "."
    os.makedirs(output_dir, exist_ok=True)

    with open(output_package_path, 'w') as output_file:
        output_file.write(merged_content)

    logging.info(f"Merged package written to {output_package_path}")
    return output_package_path

# Actual file paths
differences_file_path = "diffs/GIN_STP_PKG/procedures_gin_policies_prc_diff.txt"
heritage_package_procedure = "packages/GIN_STP_PKG_HERITAGE_BODY/procedures/gin_policies_prc.sql"
geminia_package_procedure = "packages/GIN_STP_PKG_NEW_GEMINIA_BODY/procedures/gin_policies_prc.sql"
output_package_path = "tests/merged_procedure.sql"

# Call the function to merge procedures
merged_package_path = merge_database_package_procedures_with_history(
    differences_file_path, 
    heritage_package_procedure, 
    geminia_package_procedure, 
    output_package_path
)

print(f"Merged procedure saved to: {merged_package_path}")


2024-12-23 10:17:21,650 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-23 10:17:21,905 - INFO - Generated chunk: 1 lines.
2024-12-23 10:17:21,912 - INFO - Merged package written to tests/merged_procedure.sql


Merged procedure saved to: tests/merged_procedure.sql
